# Donnez les 3 produits qui ont la plus grande valeur pour chacun des 2 ratios (vous aurez donc 6 produits à citer)

In [1]:
import pandas as pd

In [2]:
#On importe les données liées aux bilans alimentaire dans les pays confrontés à la sous-nutrition
data_sous_alimentation = pd.read_csv('fr_sousalimentation.csv')
data_vegetaux = pd.read_csv('fr_vegetaux.csv')
data_animaux = pd.read_csv('fr_animaux.csv')

#On gère nos noms de colonnes via des variables pour faciliter la maintenance du code
code_domaine = 'Code Domaine'
domaine = 'Domaine'
code_zone = "Code zone"
zone = 'Zone'
code_element = 'Code Élément'
element = 'Élément'
code_produit = 'Code Produit'
produit = 'Produit'
code_annee = 'Code année'
annee = 'Année'
unite = 'Unité'
valeur = 'Valeur'
symbole = 'Symbole'
description_symbole = 'Description du Symbole'
note = 'Note'
autres_utilisations = 'Autres utilisations (non alimentaire)'
dispo_int = 'Disponibilité intérieure'
ratio = 'ratio : Autres utilisations / Disponibilité intérieure'
aliments_animaux = 'Aliments pour animaux'
nourriture = 'Nourriture'

#On enlève les doublons et les informations inutiles de nos données
mask0 = data_vegetaux[code_zone] != 351
mask01 = data_animaux[code_zone] != 351
mask02 = data_sous_alimentation[code_zone] != 351
mask03 = data_sous_alimentation[annee] == '2012-2014'
mask04 = data_vegetaux[element] == 'Exportations - Quantité'
mask05 = data_animaux[element] == 'Exportations - Quantité'

data_vegetaux = data_vegetaux[mask0]
data_animaux = data_animaux[mask01]
data_sous_alimentation = data_sous_alimentation[mask02 & mask03]

#On gère les données non numériques, on converti nos valeurs en float et on ne garde que les pays qui souffrent de sous-nutrition
data_sous_alimentation[valeur] = data_sous_alimentation[valeur].replace('<0.1', float(0.05))
data_sous_alimentation = data_sous_alimentation.astype({valeur : 'float'})
data_sous_alimentation = data_sous_alimentation[data_sous_alimentation[valeur]>0]

data_sous_alimentation= data_sous_alimentation.drop([code_domaine, domaine, code_zone, code_element, element, code_produit, produit, code_annee, annee, unite, valeur, symbole, description_symbole, note], axis = 1)
data_vegetaux = data_vegetaux.drop([code_domaine, domaine, code_zone, code_element, code_produit, code_annee, annee, unite, symbole, description_symbole], axis = 1)
data_animaux = data_animaux.drop([code_domaine, domaine, code_zone, code_element, code_produit, code_annee, annee, unite, symbole, description_symbole], axis = 1)

#on enlève les lignes dont les données sont manquantes
data_sous_alimentation = data_sous_alimentation.dropna()

#On regroupe nos données
data_produits = data_vegetaux.append(data_animaux)
data_produits.head()

,Zone,Élément,Produit,Valeur
0,Afghanistan,Production,Blé,5169.0
1,Afghanistan,Importations - Quantité,Blé,1173.0
2,Afghanistan,Variation de stock,Blé,-350.0
3,Afghanistan,Disponibilité intérieure,Blé,5992.0
4,Afghanistan,Semences,Blé,322.0


In [3]:
data_sous_alimentation = data_sous_alimentation.groupby(zone).sum().reset_index()
data_sous_alimentation.head()

,Zone
0,Afghanistan
1,Afrique du Sud
2,Albanie
3,Algérie
4,Angola


In [4]:
#Fonction qui sert à repérer les 15 produits les plus exportés parmis les pays qui souffrent de sous-nutrition
def reperer_produits_export(data, pays) :
    data = data[data[element] == 'Exportations - Quantité']
    data = data[data[valeur] > 0]
    pays = pays[zone].values
    pays_sous_nutrition = []
    for z, p , v in zip(data[zone], data[produit], data[valeur]) :
        if z in pays :
            pays_sous_nutrition.append([p,v])
    data_pays_sous_nutrition = pd.DataFrame(pays_sous_nutrition,columns = [produit, valeur])
    data_pays_sous_nutrition = data_pays_sous_nutrition.groupby(produit).sum().sort_values(valeur, ascending = False)
    data_pays_sous_nutrition = data_pays_sous_nutrition.iloc[0 : 15, : ]
    return data_pays_sous_nutrition

In [5]:
#Fonction qui sert à trouver les 200 plus grandes importations d'un ensemble de produits
def trouver_importations(data , produits) :
    data = data[data[element] == 'Importations - Quantité']
    data = data[data[valeur] > 0]
    produits = produits[produit].values
    importations = []
    for z, p, v in zip(data[zone], data[produit], data[valeur]) :
        if p in produits :
            importations.append([z, p, v])
    data_importations = pd.DataFrame(importations, columns = [zone, produit, valeur])
    data_importations = data_importations.sort_values(valeur, ascending = False)
    data_importations = data_importations.iloc[0 : 200, : ]
    return data_importations
    

In [6]:
#Fonction qui retourne le ratio entre la quantité destinée aux 'autres utilisations' et la 'disponibilité intérieure'
def calculer_ratio_autres_util_dispo_int(data, produits) :
    mask1 = data[element] == 'Autres utilisations (non alimentaire)'
    mask2 = data[element] == 'Disponibilité intérieure'
    data = data[mask1 | mask2]
    produits = produits[produit].values
    data_info_produits = pd.DataFrame(columns =[produit, autres_utilisations, dispo_int])
    for e, p, v in zip(data[element], data[produit], data[valeur]) :
        if p in produits :
            if e == 'Autres utilisations (non alimentaire)' :
                data_info_produits.loc[len(data_info_produits.index)] = [p, v, 0]
            else :
                data_info_produits.loc[len(data_info_produits.index)] = [p, 0, v]
    
    data_info_produits = data_info_produits.groupby([produit]).sum(autres_utilisations, dispo_int)
    data_info_produits[ratio] = round(data_info_produits[autres_utilisations] / data_info_produits[dispo_int], 2)
    data_info_produits = data_info_produits.sort_values(ratio, ascending = False)
    data_info_produits = data_info_produits.iloc[0 : 3, : ]
    data_info_produits = data_info_produits.drop([autres_utilisations, dispo_int], axis = 1)
    return data_info_produits

In [7]:
#Fonction qui retourne le ratio entre la quantité destinée à la nourriture animale et la quantité destinée à la nourriture
def calculer_ratio_nourriture_animale(data, produits) :
    mask1 = data[element] == 'Aliments pour animaux'
    mask2 = data[element] == 'Nourriture'
    data = data[mask1 | mask2]
    produits = produits[produit].values
    data_info_produits = pd.DataFrame(columns =[produit, aliments_animaux, nourriture])
    for e, p, v in zip(data[element], data[produit], data[valeur]) :
        if p in produits :
            if e == 'Aliments pour animaux' :
                data_info_produits.loc[len(data_info_produits.index)] = [p, v, 0]
            else :
                data_info_produits.loc[len(data_info_produits.index)] = [p, 0, v]
    
    data_info_produits = data_info_produits.groupby([produit]).sum(aliments_animaux, nourriture)
    data_info_produits[ratio] = round(data_info_produits[aliments_animaux] / data_info_produits[nourriture], 2)
    data_info_produits = data_info_produits.sort_values(ratio, ascending = False)
    data_info_produits = data_info_produits.iloc[0 : 3, : ]
    data_info_produits = data_info_produits.drop([aliments_animaux, nourriture], axis = 1)
    return data_info_produits

In [8]:
data_exportations = reperer_produits_export(data_produits, data_sous_alimentation)
data_exportations = data_exportations.reset_index()

In [9]:
#Liste des 15 produits les plus exportés par ce groupe de pays
data_exportations.head(15)

,Produit,Valeur
0,Huile de Palme,46324.0
1,Maïs,37906.0
2,Manioc,35851.0
3,Riz (Eq Blanchi),33093.0
4,Sucre Eq Brut,26098.0
5,Blé,25342.0
6,"Légumes, Autres",20139.0
7,Bananes,17988.0
8,Lait - Excl Beurre,14973.0
9,"Fruits, Autres",14690.0


In [10]:
#Liste des 200 plus grandes importations des 15 produits les plus exportés par les pays qui souffrent de sous-nutrition
data_importations = trouver_importations(data_produits, data_exportations)
data_importations.head()

,Zone,Produit,Valeur
399,"Chine, continentale",Soja,63381.0
397,"Chine, continentale",Manioc,29046.0
937,Japon,Maïs,14403.0
527,Égypte,Blé,10331.0
1477,République de Corée,Maïs,8755.0


In [11]:
data_importations = data_importations.groupby(produit).sum(valeur).sort_values(valeur, ascending = False).reset_index()
data_importations.head(20)

,Produit,Valeur
0,Blé,147576.0
1,Maïs,102451.0
2,Soja,93455.0
3,Lait - Excl Beurre,88418.0
4,Manioc,34655.0
5,Sucre Eq Brut,34208.0
6,Huile de Palme,29747.0
7,"Légumes, Autres",28177.0
8,"Oranges, Mandarines",22945.0
9,"Fruits, Autres",15714.0


In [12]:
#Réponse à la question 12 partie 1
data_ratio_autres_util_dispo_int = calculer_ratio_autres_util_dispo_int(data_produits, data_importations)
data_ratio_autres_util_dispo_int.head(50)

,ratio : Autres utilisations / Disponibilité intérieure
Produit,
Huile de Palme,0.70
Maïs,0.20
Manioc,0.14


In [14]:
#Réponse à la question 12 partie 2
data_ratio_aliment_animaux = calculer_ratio_nourriture_animale(data_produits, data_importations)
data_ratio_aliment_animaux.head(20)

,ratio : Autres utilisations / Disponibilité intérieure
Produit,
Maïs,4.36
Soja,1.64
Manioc,0.87
